<a href="https://colab.research.google.com/github/joshmcadams/nanoGPT/blob/main/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nanoGPT

nanoGPT is a wonderful exploration into building a GPT model from scratch. We'll start with an empty notebook and quickly build and train a model to generate Shakespeare-like text.

This notebook is based off of a code and videos releasted by Andrej Karpathy. Please be sure to check out his work!

  * [Reference Video Lecture](https://www.youtube.com/watch?v=kCc8FmEb1nY)
  * [Reference Git Repository](https://github.com/karpathy/ng-video-lecture)

# Data Exploration and Preparation

## Acquiring the data

We'll first pull the data down into this lab. To get the data the `wget` command will be used in the shell.

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-23 23:28:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-03-23 23:28:13 (17.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



Next we'll load the data into Python and take a look at a sample.

First, we simply open the file and store the entire contents of the file in the variable `raw_training_data`. We get the length of the data and print that out so that we can see how much data we are dealing with.

In [17]:
with open('input.txt', 'r', encoding='utf-8') as f:
    raw_training_data = f.read()

training_data_size = len(raw_training_data)
print(training_data_size)

Next we sample the data just to get an idea of what we are dealing with. The sample comes from a random locaiton in the data. Run the code block below a few times to see some different data samples.

In [21]:
import random

SAMPLE_SIZE = 500
start = random.randrange(0, training_data_size - SAMPLE_SIZE)
print(raw_training_data[start:start+SAMPLE_SIZE])

.

PAULINA:
I pray now, call her.
Withdraw yourselves.

Gaoler:
And, madam,
I must be present at your conference.

PAULINA:
Well, be't so, prithee.
Here's such ado to make no stain a stain
As passes colouring.
Dear gentlewoman,
How fares our gracious lady?

EMILIA:
As well as one so great and so forlorn
May hold together: on her frights and griefs,
Which never tender lady hath born greater,
She is something before her time deliver'd.

PAULINA:
A boy?

EMILIA:
A daughter, and a goodly babe,
Lusty


## Data Analysis

We've now acquired the data and even poked around a bit to see what the data looks like. However, we need to dig deeper and do some more detailed analysis on the data.

One important fact to know about data that will be fed into a generative model is how many different characters are we working with. This is actually really easy in Python and can be accomplished using the `set` function.

In [28]:
tokens = sorted(list(set(raw_training_data)))
token_count = len(tokens)

print(f'There are {token_count} unique characters in the {training_data_size}' +
      f'of training data. The characters are: {"".join(tokens)}')

There are 65 unique characters in the 1115394of training data. The characters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
